In [ ]:
#  Download the required Dataset for analysis
# ! wget http://stat-computing.org/dataexpo/2009/2007.csv.bz2
# ! http://stat-computing.org/dataexpo/2009/2008.csv.bz2
# ! wget https://github.com/jayyanar/MachineLearning_Workbook/blob/master/2007-ord-weather-data.csv --no-check-certificate
# ! wget https://github.com/jayyanar/MachineLearning_Workbook/blob/master/2008-ord-weather-data.csv --no-check-certificate
# ! bzip2 -d 2007.csv.bz2
# ! bzip2 -d 2008.csv.bz2

In [ ]:
! ls -lrt


In [44]:
# For SQL-type queries (Spark)
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import udf, col, asc, desc

# For regression and other possible ML tools (Spark)
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.stat import Statistics

# Important for managing features  (Spark)
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler

# For displaying and other related IPython tools...
from IPython.display import display
from IPython.html.widgets import interact

# Typycal Python tools
import sys
import numpy as np
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import os.path

In [2]:
# To show plots inline
get_ipython().magic(u'matplotlib inline')

# function to read HDFS file into dataframe using PyDoop
import pydoop.hdfs as hdfs
def read_csv_from_hdfs(path, cols, col_types=None):
  files = hdfs.ls(path);
  pieces = []
  for f in files:
    fhandle = hdfs.open(f)
    pieces.append(pd.read_csv(fhandle, names=cols, dtype=col_types))
    fhandle.close()
  return pd.concat(pieces, ignore_index=True)

# read 2007 year file
cols = ['year', 'month', 'day', 'dow', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'Carrier', 'FlightNum', 
        'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 
        'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 
        'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay'];
flt_2007 = read_csv_from_hdfs('airline/delay/2007.csv', cols)

flt_2007.shape

In [3]:
textFile = sc.textFile('2007.csv')

In [4]:
textFileRDD = textFile.map(lambda x: x.split(','))
header = textFileRDD.first()

textRDD = textFileRDD.filter(lambda r: r != header)

In [5]:
num_records = textFileRDD.count()
print ('Number of records ' , num_records)

Number of records  7453216


In [6]:
aux_ = textFileRDD.take(2)
feature_names = aux_[0]
feature_example = aux_[1]

In [7]:
print ('Feature Names ' , feature_names)

Feature Names  ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 'UniqueCarrier', 'FlightNum', 'TailNum', 'ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'ArrDelay', 'DepDelay', 'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 'Cancelled', 'CancellationCode', 'Diverted', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']


In [8]:
print ('Feature Example ' , feature_example)

Feature Example  ['2007', '1', '1', '1', '1232', '1225', '1341', '1340', 'WN', '2891', 'N351', '69', '75', '54', '1', '7', 'SMF', 'ONT', '389', '4', '11', '0', '', '0', '0', '0', '0', '0', '0']


In [9]:
print ("Number of features = " , len(feature_example))

Number of features =  29


In [10]:
print (type(feature_example))

<class 'list'>


In [11]:
# ### Creating a SQL Dataframe from RDD
# 
# We now create a SQL DataFrame, this entity is a distributed collection of data organized into named columns. It is conceptually equivalent to a table in a relational database or a data frame in Python, but with richer optimizations under the hood. We will utilize the recently created Spark RDD and use the Spark SQL context to create the desired data frame,

# We first create function that will allow to parse a record of our RDD into the desired format. As a reference we take a look at features_names and feature_example we just created above



def parse(r):
    try:
        x=Row(Year=int(r[0]),\
          Month=int(r[1]),\
          DayofMonth=int(r[2]),\
          DayOfWeek=int(r[3]),\
          DepTime=int(float(r[4])), \
          CRSDepTime=int(r[5]),\
          ArrTime=int(float(r[6])),\
          CRSArrTime=int(r[7]), \
          UniqueCarrier=r[8],\
          DepDelay=int(float(r[15])),\
          Origin=r[16],\
          Dest=r[17], \
          Distance=int(float(r[18])),\
          CarrierDelay=int(float(r[24])),\
          WeatherDelay=int(float(r[25])),\
          NASDelay= int(float(r[26])),\
          SecurityDelay=int(float(r[27])),\
          LateAircraftDelay=int(float(r[28])))  
    except:
        x=None  
    return x

rowRDD = textRDD.map(lambda r: parse(r)).filter(lambda r:r != None)
airline_df = sqlContext.createDataFrame(rowRDD)

In [15]:
# We add a new column to our data frame, **DepDelayed**, a binary variable:
# - **True**, for flights that have > 15 minutes of delay
# - **False**, for flights that have <= 15 minutes of delay
# 
# We will later use **Depdelayed** as the target/label column in the classification process.


#airline_df = airline_df.withColumn('DepDelayed', airline_df['DepDelay']>15)
#airline_df = airline_df.withColumn('DepDelayed',airline_df['Origin']=="ORD")
#airline_df_ORD = airline_df.withColumn('DepDelayed', airline_df.filter((col("Origin") == "ORD") | (col("DepDelay")>15)))
airline_df_ORD = airline_df.filter((col("Origin") == "ORD"))
airline_df_ORD_15 = airline_df_ORD.withColumn('DepDelayed', airline_df_ORD['DepDelay']>15)

In [16]:
airline_df_ORD_15.take(10)

[Row(ArrTime=1359, CRSArrTime=1414, CRSDepTime=1100, CarrierDelay=0, DayOfWeek=4, DayofMonth=25, DepDelay=-8, DepTime=1052, Dest='EWR', Distance=719, LateAircraftDelay=0, Month=1, NASDelay=0, Origin='ORD', SecurityDelay=0, UniqueCarrier='XE', WeatherDelay=0, Year=2007, DepDelayed=False),
 Row(ArrTime=1811, CRSArrTime=1750, CRSDepTime=1500, CarrierDelay=0, DayOfWeek=7, DayofMonth=28, DepDelay=41, DepTime=1541, Dest='IAH', Distance=925, LateAircraftDelay=16, Month=1, NASDelay=5, Origin='ORD', SecurityDelay=0, UniqueCarrier='XE', WeatherDelay=0, Year=2007, DepDelayed=True),
 Row(ArrTime=2305, CRSArrTime=2211, CRSDepTime=2000, CarrierDelay=0, DayOfWeek=1, DayofMonth=29, DepDelay=45, DepTime=2045, Dest='CLE', Distance=316, LateAircraftDelay=45, Month=1, NASDelay=9, Origin='ORD', SecurityDelay=0, UniqueCarrier='XE', WeatherDelay=0, Year=2007, DepDelayed=True),
 Row(ArrTime=2204, CRSArrTime=2220, CRSDepTime=1900, CarrierDelay=0, DayOfWeek=3, DayofMonth=17, DepDelay=-9, DepTime=1851, Dest='EWR

In [28]:
print ("Total flights Origin at ORD Chicago Airport: " + str(airline_df_ORD_15.count()))
print (airline_df_ORD_15.groupby('DepDelayed').count().show())

Total flights Origin at ORD Chicago Airport: 358462
+----------+------+
|DepDelayed| count|
+----------+------+
|      true|105807|
|     false|252655|
+----------+------+

None


In [50]:
#airline_df_ORD_15.take(10)
airline_df_ORD_15.filter(airline_df_ORD_15.DepDelayed == True).groupby(airline_df_ORD_15.Month).count().sort(asc("Month")).show()

+-----+-----+
|Month|count|
+-----+-----+
|    1| 9783|
|    2|11002|
|    3|10074|
|    4| 8597|
|    5| 6716|
|    6| 9173|
|    7| 8375|
|    8|10039|
|    9| 5997|
|   10| 6826|
|   11| 6830|
|   12|12395|
+-----+-----+



In [54]:
airline_df_ORD_15.withColumn('Hour', airline_df_ORD_15.CRSDepTime.hour('timestamp')).show()

TypeError: 'Column' object is not callable

In [ ]:
# define hour function to obtain hour of day
def hour_ex(x): 
    h = int(str(int(x)).zfill(4)[:2])
    return h

# register as a UDF 
f = udf(hour_ex, IntegerType())

#CRSDepTime: scheduled departure time (local, hhmm)
airline_df = airline_df.withColumn('hour', f(airline_df.CRSDepTime))
airline_df.registerTempTable("airlineDF")

In [ ]:
#Exploration: What are the primary causes for flight delays
cause_delay = sqlContext.sql("SELECT sum(WeatherDelay) Weather,sum(NASDelay) NAS,sum(SecurityDelay) Security,sum(LateAircraftDelay) lateAircraft,sum(CarrierDelay) Carrier FROM airlineDF ")
df_cause_delay = cause_delay.toPandas()
df_cause_delay.head()

In [ ]:
#Exploration: Which Airports have the Most Delays
groupedDelay = sqlContext.sql("SELECT Origin, count(*) conFlight,avg(DepDelay) delay \
                                FROM airlineDF \
                                GROUP BY Origin")
df_origin = groupedDelay.toPandas()
df_origin.head()


In [ ]:
df_origin.sort_values('delay',ascending=0).head()

In [ ]:
! ls -lrta

In [ ]:
#o map each Airport to corresponding Long and Lat,load the dataset needed


#df = pd.read_csv('airports.dat', index_col=0, names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'], header=0)
#df = pd.read_csv('airports.dat', header=0, names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'])
df1= pd.read_csv('airports.dat',header=0, names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'])
#df = pd.read_csv('airports.dat', index_col=0,names = ['name', 'city', 'country','IATA','ICAO','lat','lng','alt','TZone','DST','Tz'], header=0)
df1.head()

In [ ]:
df_airports = pd.merge(df_origin, df, left_on = 'Origin', right_on = 'IATA')
df_airports.head()

In [ ]:
df_airports.sort_values('delay',ascending=0).head(10)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def zscore(x):
    return (x-np.average(x))/np.std(x)

In [ ]:
! conda create -n my_DSX-Python35-Spark --clone="/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark"
! conda install -c conda-forge basemap-data-hires=1.0.8.dev0 -y

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from pylab import rcParams
%matplotlib inline

rcParams['figure.figsize'] = (14,10)


my_map = Basemap(projection='merc',
            resolution = 'l', area_thresh = 1000.0,
            llcrnrlon=-130, llcrnrlat=22, #min longitude (llcrnrlon) and latitude (llcrnrlat)
            urcrnrlon=-60, urcrnrlat=50) #max longitude (urcrnrlon) and latitude (urcrnrlat)

my_map.drawcoastlines()
my_map.drawcountries()
my_map.drawmapboundary()
my_map.fillcontinents(color = 'white', alpha = 0.3)
my_map.shadedrelief()

# To create a color map
colors = plt.get_cmap('hot')(np.linspace(0.0, 1.0, 30))
colors=np.flipud(colors)

#----- Scatter -------
countrange=max(df_airports['conFlight'])-min(df_airports['conFlight'])
al=np.array([sigmoid(x) for x in zscore(df_airports['delay'])])
xs,ys = my_map(np.asarray(df_airports['lng']), np.asarray(df_airports['lat']))
val=df_airports['conFlight']*4000.0/countrange

my_map.scatter(xs, ys,  marker='o', s= val, alpha = 0.8,color=colors[(al*20).astype(int)])

#----- Text -------
df_text=df_airports[(df_airports['conFlight']>60000) & (df_airports['IATA'] != 'HNL')]
xt,yt = my_map(np.asarray(df_text['lng']), np.asarray(df_text['lat']))
txt=np.asarray(df_text['IATA'])
zp=zip(xt,yt,txt)
for row in zp:
    #print zp[2]
    plt.text(row[0],row[1],row[2], fontsize=10, color='blue',)

print("Each marker is an airport.")
print("Size of markers: Airport Traffic (larger means higher number of flights in year)")
print("Color of markers: Average Flight Delay (Redder means longer delays)")

plt.show()

In [ ]:

#Exploration: Route delay
#Which Routes are typically the most delayed?
grp_rout_Delay = sqlContext.sql("SELECT Origin, Dest, count(*) traffic,avg(Distance) avgDist,\
                                    avg(DepDelay) avgDelay\
                                FROM airlineDF \
                                GROUP BY Origin,Dest")
rout_Delay = grp_rout_Delay.toPandas()

In [ ]:
df_airport_rout1 = pd.merge(rout_Delay, df, left_on = 'Origin', right_on = 'IATA')
df_airport_rout2 = pd.merge(df_airport_rout1, df, left_on = 'Dest', right_on = 'IATA')
df_airport_rout = df_airport_rout2[["Origin","lat_x","lng_x","Dest","lat_y","lng_y",\
                                    "avgDelay", "traffic"]]

In [ ]:
df_airport_rout.sort('avgDelay',ascending=0).head()

In [ ]:
rcParams['figure.figsize'] = (14,10)


my_map = Basemap(projection='merc',
            resolution = 'l', area_thresh = 1000.0,
            llcrnrlon=-130, llcrnrlat=22, #min longitude (llcrnrlon) and latitude (llcrnrlat)
            urcrnrlon=-60, urcrnrlat=50) #max longitude (urcrnrlon) and latitude (urcrnrlat)

my_map.drawcoastlines()
my_map.drawcountries()
my_map.drawmapboundary()
my_map.fillcontinents(color = 'white', alpha = 0.3)
my_map.shadedrelief()

delay=np.array([sigmoid(x) for x in zscore(df_airports["delay"])])
colors = plt.get_cmap('hot')(np.linspace(0.0, 1.0, 40))
colors=np.flipud(colors)
xs,ys = my_map(np.asarray(df_airports['lng']), np.asarray(df_airports['lat']))
xo,yo = my_map(np.asarray(df_airport_rout['lng_x']), np.asarray(df_airport_rout['lat_x']))
xd,yd = my_map(np.asarray(df_airport_rout['lng_y']), np.asarray(df_airport_rout['lat_y']))

my_map.scatter(xs, ys,  marker='o',  alpha = 0.8,color=colors[(delay*20).astype(int)])


al=np.array([sigmoid(x) for x in zscore(df_airport_rout["avgDelay"])])
f=zip(xo,yo,xd,yd,df_airport_rout['avgDelay'],al)
for row in f:
    plt.plot([row[0],row[2]], [row[1],row[3]],'-',alpha=0.07, \
             color=colors[(row[5]*30).astype(int)] )
    

for row in zp:
    plt.text(row[0],row[1],row[2], fontsize=10, color='blue',)

print("Each line represents a route from the Origin to Destination airport.")
print("The redder line, the higher probablity of delay.")
    
plt.show()

In [ ]:
#Exploration: Airport Origin delay per month
Origin_Airport="SJC"

In [ ]:
df_ORG = sqlContext.sql("SELECT * from airlineDF WHERE origin='"+ Origin_Airport+"'")
df_ORG.registerTempTable("df_ORG")
df_ORG.select('ArrTime','CRSArrTime','CRSDepTime',\
              'DayOfWeek','DayofMonth','DepDelay','DepTime','Dest').show(2)

In [ ]:
print ("total flights from this ariport: " + str(df_ORG.count()))

In [ ]:
grp_carr = sqlContext.sql("SELECT  UniqueCarrier,month, avg(DepDelay) avgDelay from df_ORG \
                            WHERE DepDelayed=True \
                            GROUP BY UniqueCarrier,month")
s = grp_carr.toPandas()

In [ ]:
ps = s.pivot(index='month', columns='UniqueCarrier', values='avgDelay')[['AA','UA','US']]

In [ ]:
rcParams['figure.figsize'] = (8,5)
ps.plot(kind='bar', colormap='Greens');
plt.xlabel('Average delay')
plt.ylabel('Month')
plt.title('How much delay does each carrier has in each month?')

In [ ]:

#We see that average delay in this year is is highest in November and October in this airport.
#Exploration: Airport Origin delay per day/hour
hour_grouped = df_ORG.filter(df_ORG['DepDelayed']).select('DayOfWeek','hour','DepDelay').groupby('DayOfWeek','hour').mean('DepDelay')

In [ ]:
#Modeling: Logistic Regression
df_model=df_ORG
stringIndexer1 = StringIndexer(inputCol="Origin", outputCol="originIndex")
model_stringIndexer = stringIndexer1.fit(df_model)
indexedOrigin = model_stringIndexer.transform(df_model)
encoder1 = OneHotEncoder(dropLast=False, inputCol="originIndex", outputCol="originVec")
df_model = encoder1.transform(indexedOrigin)

In [ ]:
assembler = VectorAssembler(
    inputCols = ['Year','Month','DayofMonth','DayOfWeek','hour','Distance','originVec'],
    outputCol = "features")
output = assembler.transform(df_model)
airlineRDD=output.map(lambda row: LabeledPoint([0,1][row['DepDelayed']],row['features']))

In [ ]:
#  Spliting dataset into train and test dtasets
trainRDD,testRDD=airlineRDD.randomSplit([0.7,0.3])

In [ ]:
# Build the model
model = LogisticRegressionWithLBFGS.train(trainRDD)

In [ ]:
#Model Evaluation
# Evaluating the model on testing data
labelsAndPreds = testRDD.map(lambda p: (p.label, model.predict(p.features)))

In [ ]:
def conf(r):
    if r[0] == r[1] ==1: x= 'TP'
    if r[0] == r[1] ==0: x= 'TN'
    if r[0] == 1 and  r[1] ==0: x= 'FN'
    if r[0] == 0 and  r[1] ==1: x= 'FP'
    return (x)
acc1 = labelsAndPreds.map(lambda (v, p): ((v, p),1)).reduceByKey(lambda a, b: a + b).take(5)
acc = [(conf(x[0]),x[1]) for x in acc1]

In [ ]:
TP=TN=FP=FN=0.0
for x in acc: 
    if x[0]=='TP': TP= x[1]
    if x[0]=='TN': TN= x[1]
    if x[0]=='FP': FP= x[1]
    if x[0]=='FN': FN= x[1]
eps = sys.float_info.epsilon
Accuracy = (TP+TN) / (TP + TN+ FP+FN+eps) 
print ("Model Accuracy for JFK: %1.2f %%" % (Accuracy*100))